In [ ]:
import plot
import classify
import importlib
import preprocess
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# reload when updating code
importlib.reload(plot)
importlib.reload(classify)
importlib.reload(preprocess)
# mouse and date
mouse = 'NN8'
date = '210316'
# create folders to save files
paths = preprocess.create_folders(mouse, date)
# import data for mouse and date as dict
session_data = preprocess.load_data(paths)
# process and plot behavior
behavior = preprocess.process_behavior(session_data, paths)
# save masks so can run in matlab to process other planes
preprocess.cell_masks(paths, 0)

In [ ]:
# reload when updating code
importlib.reload(plot)
importlib.reload(classify)
importlib.reload(preprocess)
# grab activity 
deconvolved = preprocess.process_activity(paths, 'spks', 3, 0)
# normalize activity
norm_deconvolved = preprocess.normalize_deconvolved(deconvolved, behavior, paths, 0)
# gassuain filter acitivity
norm_moving_deconvolved_filtered = preprocess.difference_gaussian_filter(norm_deconvolved, 4, behavior, paths, 0)
# make trial averaged traces and basline subtract
mean_cs_1_responses_df = preprocess.normalized_trial_averaged(norm_deconvolved, behavior, 'cs_1')
mean_cs_2_responses_df = preprocess.normalized_trial_averaged(norm_deconvolved, behavior, 'cs_2')
# get sig cells
[cs_1_poscells, cs_1_negcells] = preprocess.sig_test(norm_deconvolved, behavior, 'cs_1')
[cs_2_poscells, cs_2_negcells] = preprocess.sig_test(norm_deconvolved, behavior, 'cs_2')
[both_poscells, both_sigcells] = preprocess.combine_sig(cs_1_poscells, cs_1_negcells, cs_2_poscells, cs_2_negcells)
# get idx of top cell differences
idx = preprocess.get_index(behavior, mean_cs_1_responses_df, mean_cs_2_responses_df, cs_1_poscells, cs_2_poscells, both_poscells, both_sigcells, paths, 0)

In [ ]:
# reload when updating code
importlib.reload(plot)
importlib.reload(classify)
importlib.reload(preprocess)
# get prior for synchronous cue activity
prior = classify.prior(norm_moving_deconvolved_filtered, idx['cs_1'], idx['cs_2'], behavior, [])
# logistic regression
y_pred = classify.log_regression(behavior, norm_deconvolved, norm_moving_deconvolved_filtered, both_poscells, prior)
# process classified output
y_pred = classify.process_classified(y_pred, prior, paths, 0)

In [ ]:
# reload when updating code
importlib.reload(plot)
importlib.reload(classify)
importlib.reload(preprocess)
plot.reactivation_cue_vector(norm_deconvolved, idx, y_pred, behavior, paths, [], [])

In [ ]:
# reload when updating code
importlib.reload(plot)
importlib.reload(classify)
importlib.reload(preprocess)
# plot heatmap of top cells
plot.sorted_map(behavior, mean_cs_1_responses_df, mean_cs_2_responses_df, idx['cs_1'], idx['cs_2'], 150, paths)
# plot mean reactivation probability after cues
plot.mean_reactivation(y_pred, behavior, paths, [], [])
# plot reactivation bias over time
plot.all_reactivation_bias(y_pred, behavior, paths, [], [])
# plot physical evoked reactivations
plot.reactivation_physical(y_pred, behavior, paths, [], [])
# plot activity change with reactivation rates over time
plot.activity_across_trials(norm_deconvolved, norm_moving_deconvolved_filtered, behavior, y_pred, idx, paths, [], [])
# plot reactivation raster
plot.reactivation_raster(behavior, norm_deconvolved, y_pred, idx['cs_1_df'], idx['cs_2_df'], idx['both'], paths, [])
#plot.sample_reactivation_raster(behavior, norm_deconvolved, y_pred, idx['cs_1_df'], idx['cs_2_df'], idx['both'], paths, 46500, 47500)

In [ ]:
# reload when updating code
importlib.reload(plot)
importlib.reload(classify)
importlib.reload(preprocess)
# create folders to save files
paths = preprocess.create_folders(mouse, date)
# import data for mouse and date as dict
session_data = preprocess.load_data(paths)
# process and plot behavior
behavior = preprocess.process_behavior(session_data, paths)
# normalize activity
norm_deconvolved = preprocess.normalize_deconvolved([], behavior, paths, 0)
# Gaussian filter activity
norm_moving_deconvolved_filtered = preprocess.difference_gaussian_filter(norm_deconvolved, 4, behavior, paths, 0)

In [ ]:
# reload when updating code
importlib.reload(plot)
importlib.reload(classify)
importlib.reload(preprocess)
import random
# random subset of cells
rand_cells = random.sample(range(0, len(norm_deconvolved)), int(len(norm_deconvolved)/10))
norm_deconvolved_subset = norm_deconvolved.loc[rand_cells, :]
norm_moving_deconvolved_filtered_subset = norm_moving_deconvolved_filtered[rand_cells, :]
# make trial averaged traces and baseline subtract
mean_cs_1_responses_df = preprocess.normalized_trial_averaged(norm_deconvolved_subset, behavior, 'cs_1')
mean_cs_2_responses_df = preprocess.normalized_trial_averaged(norm_deconvolved_subset, behavior, 'cs_2')
# get sig cells
[cs_1_poscells, cs_1_negcells] = preprocess.sig_test(norm_deconvolved_subset, behavior, 'cs_1')
[cs_2_poscells, cs_2_negcells] = preprocess.sig_test(norm_deconvolved_subset, behavior, 'cs_2')
[both_poscells, both_sigcells] = preprocess.combine_sig(cs_1_poscells, cs_1_negcells, cs_2_poscells,
                                                        cs_2_negcells)
# get idx of top cell differences
idx = preprocess.get_index(behavior, mean_cs_1_responses_df, mean_cs_2_responses_df, cs_1_poscells,
                           cs_2_poscells, both_poscells, both_sigcells, paths, 2)
# get idx of top cell differences
idx_original = preprocess.get_index(behavior, mean_cs_1_responses_df, mean_cs_2_responses_df, cs_1_poscells, cs_2_poscells, both_poscells, both_sigcells, paths, 0)
# get prior for synchronous cue activity
prior = classify.prior(norm_moving_deconvolved_filtered_subset, idx['cs_1'], idx['cs_2'], behavior, [])
# logistic regression
y_pred = classify.log_regression(behavior, norm_deconvolved_subset, norm_moving_deconvolved_filtered_subset,
                                 both_poscells, prior)
# process classified output
y_pred = classify.process_classified(y_pred, prior, paths, 2)
y_pred_original = classify.process_classified([], [], paths, 0)

import pandas as pd
import numpy as np
import plot_subset
importlib.reload(plot_subset)
plot_subset.sample_reactivation_raster(behavior, pd.DataFrame(np.array(norm_deconvolved_subset)), norm_deconvolved, y_pred, y_pred_original, 
                                       idx['cs_1_df'], idx['cs_2_df'], idx_original['cs_1_df'], idx_original['cs_2_df'], paths, 24500, 25500)